# Leveraging Confidence Intervals for Effective Stock Management

The main purpose of this project is to demonstrate a useful way confidence intervals can be effectively utilized for enhacing a company's inventory management for specific future periods. Confidence intervals are applied to the same dataset for three different future periods and the results are compared with each other. Visualizations are provided to better highlight the differences.

## Environment Management

Please read the project's README for instructions on how to set up the project's environment on your computer.

In [ ]:
import sys
sys.executable

## Project Libraries

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Import the Dataset

In [ ]:
filepath = "original.xlsx"
sheetname = "Al Bundy"
# Read specific sheet from the Excel file and skip the first 3 rows:
df = pd.read_excel(filepath, sheet_name=sheetname, skiprows=3)
df.head()

## Data Cleaning

In [ ]:
print(df.info())
df.head()

In [ ]:
# Delete the unecessary columns:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 12", "Month", "Size (US)", "Size (UK)"])

# Truncate the datetime to the nearest day, time is unecessary:
df["Date"] = df["Date"].dt.floor("D")

In [ ]:
# Define a function which converts the shoe sizes to single decimal values:
def convert_to_decimal(size):
    # Make sure the if statement will work treating the input as a strings:
    size = str(size)
    # Check if the size is a range, instead of a single numerical value:
    if "-" in size:
        # Split this range into low and high values:
        low, high = size.split("-")
        # Return a single decimal value adding 0.5 at the low value:
        return float(int(low) + 0.5)
    else:
        # If not a range, covert the size to a float and return it:
        return float(size)

# Apply the above function to the "Size (Europe)" column:
df["Size (Europe)"] = df["Size (Europe)"].apply(convert_to_decimal)

In [ ]:
df.head()

This is the final, cleaned df.

## Descriptive Statistics

In [ ]:
# Group by "Gender" keeping specific columns:
gender_group_df = df.groupby("Gender")[["Date", "Size (Europe)", "UnitPrice", "Discount", "SalePrice"]]

# Create a male df:
df_male = gender_group_df.get_group("Male")

# Create a female df:
df_female = gender_group_df.get_group("Female")

# Apply describe method to the grouped dfs:
summary_stats = gender_group_df.describe().transpose()

summary_stats

In [ ]:
# See the correlations between the columns:
gender_group_df.corr()

In [ ]:
# Ignore the warnings:
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")

# Create a figure:
plt.figure(figsize=(10, 6))

# Create histograms for shoe sizes for males and females:
sns.histplot(
    data=df_male, 
    x="Size (Europe)", 
    alpha=0.4, 
    bins=np.arange(39, 48, 0.5), 
    color="blue", 
    label="Male"
)
sns.histplot(
    data=df_female, 
    x="Size (Europe)", 
    alpha=0.4, 
    bins=np.arange(35, 42.5, 0.5), 
    color="red", 
    label="Female"
)

# Set the title and labels:
plt.title("Histogram of Shoe Sizes Sales by Gender")
plt.xlabel("Shoe Size (Europe)")
plt.ylabel("Frequency - Sales")

# Set x-axis to indicate the actual intervals: 
plt.xticks(np.arange(35, 48, 0.5), rotation=45)

# Show the legend:
plt.legend();

plt.savefig('histogram', dpi=300, bbox_inches='tight')

Based on the above histograms, we can assume normality. This provides a good starting point for proceeding to the next part, which concerns the creation of confidence intervals. 

## Inferential Statistics  

This section addresses the main goal of the project. Let's discuss about the dataset ensuring we can make accurate predictions. 

The dataset includes 3 years of sales data. This is the sample size for the future predictions. This sample dictates the maximum future period we can forecast, that is, 3 years. However, aiming for a 3-year sale prediction, without multiple 3-year samples is pointless. It's more logical to predict the sales for shorter future periods, such as, 1 year, 3 months and 1 month. These approaches have different sample sizes with $n=3$, $n=12$ and $n=36$ respectively. 

Since the dataset spans 3-year sales, this is a large sample (Al Bundy's operates for more than 30 years) which allows for the application of the Central Limit Theorem. Consequently, we can assume normality in the data and proceed with the confidence intervals calculations.

### Data Preprocessing before the Application of Confidence Intervals

In [ ]:
# Create three copies of the original df, one copy for each approach for males:
df_year_male = df_male.copy()
df_quarter_male = df_male.copy()
df_month_male = df_male.copy()

# Adapt the datetimes to each approach:
# 1-year adaption
df_year_male["Date (year)"] = df_year_male["Date"].dt.year

# Delete the old date column:
df_year_male = df_year_male.drop(columns="Date")

# 3-month adaption
df_quarter_male["Quarter"] = (
    df_quarter_male["Date"].dt.quarter
)  # Keep the quarter of each datetime to a new column.
df_quarter_male["Year"] = (
    df_quarter_male["Date"].dt.year
)  # Keep the year of each datetime to a new column.

# Concatenate the quarter with the year columns:
df_quarter_male["Date (quarter)"] = (
    df_quarter_male["Quarter"].astype(str) + "-" + df_quarter_male["Year"].astype(str)
)

# Delete the rest of the columns:
df_quarter_male = df_quarter_male.drop(columns=["Quarter", "Year", "Date"])

# 1-month adaption
df_month_male["Month"] = (
    df_month_male["Date"].dt.month
)  # Keep the month of each datetime to a new column.
df_month_male["Year"] = (
    df_month_male["Date"].dt.year
)  # Keep the year of each datetime to a new column.

# Concatenate the month with the year columns:
df_month_male["Date (month)"] = (
    df_month_male["Month"].astype(str) + "-" + df_month_male["Year"].astype(str)
)

# Delete the rest of the columns:
df_month_male = df_month_male.drop(columns=["Month", "Year", "Date"])

We wont repeat the same process for women's population because the results will be similar.

### Confidence Intervals for 1-year Sales Prediction for Males

##### Key Considerations
There is one population with an unknown variance and normality can be assumed.

##### Calculation of Mean and Standard Deviation per Shoe Size Combining all Years

In [ ]:
# Create a df from the shoe size value_counts series:
male_sales_per_year = (
    df_year_male.groupby(["Date (year)", "Size (Europe)"]).size().reset_index(name="Sales")
)

# Calculate the mean and the std per shoe size combining all three years:
combined_sales_per_year = (
    male_sales_per_year.groupby("Size (Europe)")["Sales"].agg(["mean", "std"]).reset_index()
)

# Set a clearer column names:
combined_sales_per_year.columns = ["Size (Europe)", "Sales Mean", "Sales Standard Deviation"]

In [ ]:
# Assign some columns in variables, gaining space afterwards:
sizes = combined_sales_per_year["Size (Europe)"]
means = combined_sales_per_year["Sales Mean"]
std = combined_sales_per_year["Sales Standard Deviation"]

##### Calculation of Standard Error for each Shoe Size
This is the formula for the standard error of the mean:
$$SE_\bar{x}=\frac{s}{\sqrt{n}}$$
Where, $s$ is the sample standard deviation and $n$ is the sample size. The sample size, $n$ is 3, then the degrees of freedom, $df$ is 2.

In [ ]:
# Assign the sample size, n value to a variable:
n = 3

# Add a new column containing sample size, n, per shoe size:
combined_sales_per_year["n"] = n

# Add a new column calculating the standard error, SE, per shoe size:
combined_sales_per_year["Standard Error"] = std / (combined_sales_per_year["n"]**0.5)

print(combined_sales_per_year)

In [ ]:
# Assign some columns in variables, gaining space afterwards:
st_error = combined_sales_per_year["Standard Error"]

##### Calculation of t-statistic
Assuming we want to be 95%  confident the next year's sales per shoe size will fall inside our confident interval range, the confident level is $1-α=0.95$, and hence $α=0.05$. The degrees of freedom, $n-1$ equals $3-1=2$, $df=2$. Therefore, we are looking for $t_{2, 0.025}$. From t-distribution table critical values table we can see that $t_{2, 0.025}=4.303$.

In [ ]:
# Assign the t-statistic value to a variable:
t_statistic = 4.303

# Create a new column with the margin of error:
combined_sales_per_year["Margin of Error"] = t_statistic * st_error

In [ ]:
# Assign some columns in variables, gaining space afterwards:
mrg_error = combined_sales_per_year["Margin of Error"]

##### Calculation of Confidence Intervals

In [ ]:
# Create a new column containing the lower confidence interval for each shoe size:
combined_sales_per_year["Lower CI"] = means - mrg_error

# Create a new column containing the upper confidence interval for each shoe size:
combined_sales_per_year["Upper CI"] = means + mrg_error

combined_sales_per_year

In [ ]:
# Create the figure:
plt.figure(figsize=(12, 8))

# Create an errorbar of sizes VS sales mean with errorbars displayed in the y-direction:
plt.errorbar(sizes, means, yerr=mrg_error, fmt="o-", capsize=7, color="skyblue", ecolor="red")

# Name axis and figure:
plt.xlabel("Shoe Size (Europe)")
plt.ylabel("Sales Mean per Year")
plt.title("Sales Mean per Year with 95% Confidence Intervals per Shoe Size")

# Set the tickers and create a grid:
plt.xticks(sizes, rotation=45)
plt.yticks(np.arange(-150, 1400, 100))
plt.grid(axis="y", linestyle="--");

plt.savefig('errorbar', dpi=300, bbox_inches='tight')

##### Conclusions
The results indicate that trying to predict next year's sales to control inventory management is challenging due to huge variability of the data. The wide confidence intervals suggests that sales could vary significantly making precise predictions difficult. Therefore, we 'll try another approach increasing the sample size in order to reduce the margin of error and provide more accurate estimates. 

### Confidence Intervals for 3-month Sales Prediction for Males

##### Key Considerations
There is one population with an unknown variance and normality can be assumed.

##### Calculation of Mean and Standard Deviation per Shoe Size Combining all Quarters

In [ ]:
# Create a df from the shoe size value_counts series:
male_sales_per_quarter = (
    df_quarter_male.groupby(["Date (quarter)", "Size (Europe)"]).size().reset_index(name="Sales")
)

# male_sales_per_quarter should produce 180 results (15 shoe sizes * 4 quarters * 3 years).
# However, there wasn't any sale of 46 shoe size during the first quarter of 2014 (1-2014).
# Create the missing observation:
missing_observation = pd.DataFrame([{"Date (quarter)": "1-2014", "Size (Europe)": 46.0, "Sales": 0}])

# Append the missing observation to the df:
male_sales_per_quarter = pd.concat([male_sales_per_quarter, missing_observation], ignore_index=True)

# Sort by "Date (quarter)" and "Size (Europe)" to maintain order:
male_sales_per_quarter = (
    male_sales_per_quarter.sort_values(["Date (quarter)", "Size (Europe)"]).reset_index(drop=True)
)

# Calculate the mean and the std per shoe size combining all three years:
combined_sales_per_quarter = (
    male_sales_per_quarter.groupby("Size (Europe)")["Sales"].agg(["mean", "std"]).reset_index()
)

# Set a clearer column names:
combined_sales_per_quarter.columns = ["Size (Europe)", "Sales Mean", "Sales Standard Deviation"]

In [ ]:
# Assign some columns in variables, gaining space afterwards:
sizes = combined_sales_per_quarter["Size (Europe)"]
means = combined_sales_per_quarter["Sales Mean"]
std = combined_sales_per_quarter["Sales Standard Deviation"]

##### Calculation of Standard Error for each Shoe Size

In [ ]:
# Assign the sample size, n to a variable:
n = 12

# Add a new column containing sample size, n, per shoe size:
combined_sales_per_quarter["n"] = n

# Add a new column containing the standard error, SE, per shoe size:
combined_sales_per_quarter["Standard Error"] = std / (combined_sales_per_quarter["n"]**0.5)

print(combined_sales_per_quarter)

In [ ]:
# Assign some columns in variables, gaining space afterwards:
st_error = combined_sales_per_quarter["Standard Error"]

##### Calculation of t-statistic
Assuming we want to be 95% confident the next quarter's sales per shoe size will fall inside our confident interval range, the confident level is $1-α=0.95$, and hence $α=0.05$. The degrees of freedom, $n-1$ equals $12-1=11$, $df=11$. Therefore, we are looking for $t_{11, 0.025}$. From t-distribution table critical values table we can see that $t_{11, 0.025}=2.201$.

In [ ]:
# Assign the t-statistic value to a variable:
t_statistic = 2.201

# Create a new column with the margin of error:
combined_sales_per_quarter["Margin of Error"] = t_statistic * st_error

In [ ]:
# Assign some columns in variables, gaining space afterwards:
mrg_error = combined_sales_per_quarter["Margin of Error"]

##### Calculation of Confidence Intervals

In [ ]:
# Create a new column containing the lower confidence interval for each shoe size:
combined_sales_per_quarter["Lower CI"] = means - mrg_error

# Create a new column containing the upper confidence interval for each shoe size:
combined_sales_per_quarter["Upper CI"] = means + mrg_error

combined_sales_per_quarter

In [ ]:
# Create the figure:
plt.figure(figsize=(12, 8))

# Create an errorbar of sizes VS sales mean with errorbars displayed in the y-direction:
plt.errorbar(sizes, means, yerr=mrg_error, fmt="o-", capsize=7, color="skyblue", ecolor="red")

# Name axis and figure:
plt.xlabel("Shoe Size (Europe)")
plt.ylabel("Sales Mean per Quarter")
plt.title("Sales Mean per Quarter with 95% Confidence Intervals per Shoe Size")

# Set the tickers and create a grid:
plt.xticks(sizes, rotation=45)
plt.yticks(np.arange(0, 225, 25))
plt.grid(axis="y", linestyle="--");

plt.savefig('errorbar2', dpi=300, bbox_inches='tight')

##### Conclusions
The CIs for the quarterly data are narrower compared to the yearly data. This suggests that the increased sample size, $n=12$, has successfully reduced the margin of error, resulting in more precise estimates. However, some shoe sizes, such as 42 and 42.5 still exhibit a relatively high margin of error indicating higher variability in sales for these sizes even for a quarterly prediction. The margin of error for most sizes are small, showing that these specific estimates are more reliable. For instance, size 46 has a lower CI of 4.29 and an upper CI of 10.21, which is a very good improvement over the previous yearly examination.

### Confidence Intervals for 1-month Sales Prediction for Males Using t-statistic

##### Key Considerations
There is one population with an unknown variance and normality can be assumed.

##### Calculation of Mean and Standard Deviation per Shoe Size Combining all Months
We won't present the whole code in one cell because data modifications are necessary for this approach. Specifically, the df below ("males_sales_per_month") should yield 540 results (15 shoe sizes * 12 months * 3 years). However, there are only 514 results, meaning there are zero sales of specific sizes at certain months. These zero sales are treated as missing values by Pandas. Therefore, we have to address this issue before proceeding further.

In [ ]:
# Create a df from the shoe size value_counts series:
male_sales_per_month = (
    df_month_male.groupby(["Date (month)", "Size (Europe)"])
    .size()
    .reset_index(name="Sales")
)

print(len(male_sales_per_month))

One solution to this problem is to create a pivot table df, setting "Date (month)" as an index, the value_counts of "Size (Europe)" as columns, "Sales" as the df values, and finally filling the missing values with zeros.

In [ ]:
# Create the pivot table as described above filling the missing values with zeros:
dummy_table = (
    male_sales_per_month.pivot_table(
        index="Date (month)", columns="Size (Europe)", values="Sales", fill_value=0
    )
)

In [ ]:
# Reset index to make "Date (month)" a column again:
male_sales_per_month = dummy_table.reset_index()

To revert the pivot process, we 'll apply the melt() method to unpivot the df and making it as it originally was. The variable which should be converted is "Size (Europe)" while we need to add all "Sales" values under the "Size (Europe)" columns and display the result into a single column. Additionally, "Date (month)" should play the role of the identifier column.

In [ ]:
# Unpivot the df converting columns into rows:
male_sales_per_month = (
    male_sales_per_month.melt(
        id_vars="Date (month)", var_name="Size (Europe)", value_name="Sales"
    )
)

In [ ]:
# Sort the DataFrame by month and shoe size and fix the index:
male_sales_per_month = (
    male_sales_per_month.sort_values(
        by=["Date (month)", "Size (Europe)"]
    )
    .reset_index(drop=True)
)

At this point, the result is exactly the same with the original one, having the missing values filled with zeros (540 observations). We can now proceed calculating the mean and the standard deviation.

In [ ]:
print(len(male_sales_per_month))

In [ ]:
# Calculate the mean and the std per shoe size combining all three years:
combined_sales_per_month = (
    male_sales_per_month.groupby("Size (Europe)")["Sales"]
    .agg(["mean", "std"])
    .reset_index()
)

# Set a clearer column names:
combined_sales_per_month.columns = ["Size (Europe)", "Sales Mean", "Sales Standard Deviation"]

In [ ]:
# Assign some columns to variables, gaining space afterwards:
sizes = combined_sales_per_month["Size (Europe)"]
means = combined_sales_per_month["Sales Mean"]
std = combined_sales_per_month["Sales Standard Deviation"]

##### Calculation of Standard Error for each Shoe Size

In [ ]:
# Assign the sample size, n to a variable:
n = 36

# Add a new column containing sample size, n, per shoe size:
combined_sales_per_month["n"] = n

# Add a new column containing the standard error, SE, per shoe size:
combined_sales_per_month["Standard Error"] = std / (combined_sales_per_month["n"]**0.5)

print(combined_sales_per_month)

In [ ]:
# Assign some columns in variables, gaining space afterwards:
st_error = combined_sales_per_month["Standard Error"]

##### Calculation of t-statistic
The sample size, $n$, equals $36$ which is greater than $30$. This implies that the standard deviations derived from these observations may effectively represent the population's standard deviations. While either the z-statistic or the t-statistic choice could yield similar results we 'll choose the t-statistic.

Assuming we want to be 95% confident the next month's sales per shoe size will fall inside our confident interval range, the confident level is $1-α=0.95$, and hence $α=0.05$. The degrees of freedom, $n-1$ equals $36-1=35$, $df=35$. Therefore, we are looking for $t_{35, 0.025}$. From t-distribution table critical values table we can see that $t_{35, 0.025}=2.03$, while the corresponding z-statistic value is $1.96$.

In [ ]:
# Assign the t-statistic value to a variable:
t_statistic = 2.03

# Create a new column with the margin of error:
combined_sales_per_month["Margin of Error"] = t_statistic * st_error

In [ ]:
# Assign some columns in variables, gaining space afterwards:
mrg_error = combined_sales_per_month["Margin of Error"]

##### Calculation of Confidence Intervals

In [ ]:
# Create a new column containing the lower confidence interval for each shoe size:
combined_sales_per_month["Lower CI"] = means - mrg_error

# Create a new column containing the upper confidence interval for each shoe size:
combined_sales_per_month["Upper CI"] = means + mrg_error

combined_sales_per_month

In [ ]:
# Create the figure:
plt.figure(figsize=(12, 8))

# Create an errorbar of sizes VS sales mean with errorbars displayed in the y-direction:
plt.errorbar(sizes, means, yerr=mrg_error, fmt="o-", capsize=7, color="skyblue", ecolor="red")

# Name axis and figure:
plt.xlabel("Shoe Size (Europe)")
plt.ylabel("Sales Mean per Month")
plt.title("Sales Mean per Month with 95% Confidence Intervals per Shoe Size")

# Set the tickers and create a grid:
plt.xticks(sizes, rotation=45)
plt.yticks(np.arange(0, 60, 5))
plt.grid(axis="y", linestyle="--");

plt.savefig('errorbar3', dpi=300, bbox_inches='tight')

##### Conclusions
Compared to the yearly and quarterly data, the monthly data shows significantly narrower confidence intervals. This indicates a higher precision in estimating the next month's sales for each shoe size. The increased sample size (n = 36) has effectively reduced the margin of error, leading to more reliable estimates. For example, size 42.5 has a lower CI of 45.17 and an upper CI of 59.83, which is much narrower compared to previous intervals.

### Confidence Intervals for 1-month Sales Prediction for Males Using z-statistic
Assuming the sample size is sufficiently large to accurate give the true standard deviations, then $σ=s$. Based on this assumption, the process of calculating the CIs using the z-statistic should be similar to the corresponding one, using the t-statistic. Let's examine the results.

In [ ]:
# Assign the z-statistic value to a variable:
z_statistic = 1.96

# Create a new column with the margin of error:
combined_sales_per_month["Margin of Error (z-stat)"] = z_statistic * st_error

In [ ]:
# Assign some columns in variables, gaining space afterwards:
mrg_error = combined_sales_per_month["Margin of Error (z-stat)"]

In [ ]:
# Create a new column containing the lower confidence interval for each shoe size:
combined_sales_per_month["Lower CI (z-stat)"] = means - mrg_error

# Create a new column containing the upper confidence interval for each shoe size:
combined_sales_per_month["Upper CI (z-stat)"] = means + mrg_error

combined_sales_per_month

##### Conclusions for Monthly Sales Using the z-statistic
The obtained results are almost equal to those derived from the t-statistic approach, as anticipated. While both methods yield very similar results, we 'll avoide making the additional assumption required for applying z-statistic, and hence we 'll pick t-statistic approach.

## Inventory Management Conclusions
Assuming I am the manager of Al Bundy's owing this exact dataset, my primary focus would be the monthly sales data. However, taking into account only the monthly data has a risk of information overload. For example, having some customers who buy exceptionally large stocks only once or twice per year might lead to inventory management problems during those months. Therefore, it would be crucial to "see" less information adopting a quarterly or 6-month approach in order to better "catch" these large stocks trends. Nevertheless, assuming that there are limitations over the purchase of excessively large quantities of shoes, the monthly sales data are a good option for effective inventory management. The upper limits of monthly sales confidence intervals serve as reliable production targets, ensuring the store maintains enough stock levels and minimizes the risk of losing a sale due to zero inventory of specific shoe models.